In [233]:
import requests
import time
import pickle
import json
import datetime
import pandas as pd
from stop_words import get_stop_words
from pprint import pprint
# lemmatization
import nltk
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation
#postagging
import pymorphy2
from collections import Counter
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
# plotting tools
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
owner_id = '-34215577'
domain = 'overhear'
count = 100
offset = 0 #по началу offset=0, в цикле мы его будем смещать на 100
access_token = 'b25a4074b25a4074b25a4074b0b22f7ffabb25ab25a4074ed9c6b337b5e719a2f5bcbc3'

In [3]:
def getjson(url, data=None):
    response = requests.get(url, params=data)
    print(response.url) #смотрим, правильно ли сформировался url
    return response.json()

In [4]:
#параметры count и offset задаем дефолтно, при использовании функции прописывать не будем
#они нужны на случай, если придется поменять

def get_all_posts(owner_id, domain, access_token, count = 100, offset = 0):
    '''takes in owner_id(group id), domain(group domain), count(default=100),
    offset(default=0) and returns all posts from vk group in a list of
    dictionaries in the first variable and the number of posts in a second variable'''
    
    all_posts = []
    
    while True:
    #учитываем частотные ограничения: каждый запрос будет происходить с задержкой в секунду
        time.sleep(1)
        wall = getjson('https://api.vk.com/method/wall.get', {
            'owner_id':owner_id,
            'domain':domain, 
            'count':count, 
            'offset':offset, 
            'access_token':access_token, 
            'v':'5.126'
            })
        
        count_posts = wall['response']['count']
        posts = wall['response']['items']
        
        #расширяем список записями, которые приходят из цикла
        all_posts.extend(posts)
        
        if len(all_posts) >= count_posts:
            break
        else:
            offset += 100
        
    return all_posts, count_posts

In [5]:
all_posts, count_posts = get_all_posts(owner_id, domain, access_token)

https://api.vk.com/method/wall.get?owner_id=-34215577&domain=overhear&count=100&offset=0&access_token=b25a4074b25a4074b25a4074b0b22f7ffabb25ab25a4074ed9c6b337b5e719a2f5bcbc3&v=5.126
https://api.vk.com/method/wall.get?owner_id=-34215577&domain=overhear&count=100&offset=100&access_token=b25a4074b25a4074b25a4074b0b22f7ffabb25ab25a4074ed9c6b337b5e719a2f5bcbc3&v=5.126
https://api.vk.com/method/wall.get?owner_id=-34215577&domain=overhear&count=100&offset=200&access_token=b25a4074b25a4074b25a4074b0b22f7ffabb25ab25a4074ed9c6b337b5e719a2f5bcbc3&v=5.126
https://api.vk.com/method/wall.get?owner_id=-34215577&domain=overhear&count=100&offset=300&access_token=b25a4074b25a4074b25a4074b0b22f7ffabb25ab25a4074ed9c6b337b5e719a2f5bcbc3&v=5.126
https://api.vk.com/method/wall.get?owner_id=-34215577&domain=overhear&count=100&offset=400&access_token=b25a4074b25a4074b25a4074b0b22f7ffabb25ab25a4074ed9c6b337b5e719a2f5bcbc3&v=5.126
https://api.vk.com/method/wall.get?owner_id=-34215577&domain=overhear&count=100&of

In [7]:
print('Количество всех постов на странице сообщества:', count_posts)

Количество всех постов на странице сообщества: 136374


In [6]:
#сохраняем в json
#with open('podslushano_posts.json', 'w') as f:
    #json.dump(all_posts, f)

In [8]:
all_posts[:10]

[{'id': 1308574,
  'from_id': -34215577,
  'owner_id': -34215577,
  'date': 1608291001,
  'marked_as_ads': 0,
  'post_type': 'post',
  'text': 'В «Подслушано» открыт эксклюзивный закрытый клуб 🤪 \n \nМы ищем Донов. Стань Доном и войди в ряды людей с супер способностями. К примеру, Доны могут комментировать секреты и читать закрытые от посторонних глаз записи в «Подслушано». \n \nДобро пожаловать! \n \nПодробнее в блоке «Поддержать автора» внутри [club34215577|нашего сообщества].',
  'is_pinned': 1,
  'attachments': [{'type': 'photo',
    'photo': {'album_id': -7,
     'date': 1608247398,
     'id': 457287837,
     'owner_id': -34215577,
     'has_tags': False,
     'access_key': 'a31044b5fe7bed0f50',
     'post_id': 1308574,
     'sizes': [{'height': 77,
       'url': 'https://sun3-11.userapi.com/impg/y2quMW5590OU_SzJ1sSCsm4ATqbBu5M8uBhHjg/On1RYYgjnFs.jpg?size=130x77&quality=96&sign=ab3df31a0fb1b9abeb5e63416e02d4a0&c_uniq_tag=aNxsRXD0CuEAU1adHFIk3fdEyl3st7aViSUkh2gJnSQ&type=album',
   

In [9]:
#оставляем нужные теги
def clean_posts(all_posts):
    '''takes in a list of dictionaries with posts, converts the data in a new structure 
    and returns a new list with dictionaries with the posts'''
    
    cleaned_data = []
    for post in all_posts:
        #try нужен, так как в списке словарей может не быть некоторых параметров (где-то нет текста, где-то нет даты)
        #дата в timestamp формате - это линуксовский формат
        #форматируем timestamp, приводим к целочисленному значению, если строка.
        try:
            date = datetime.datetime.fromtimestamp(int(post['date'])).strftime('%d-%m-%Y %H:%M:%S')
        except:
            date = '' #если будет ошибка, даты не будет, оставляем пустое поле 
        try:
            timestamp = post['date'] #оставляем в линуксовском формате
        except:
            timestamp = ''
        
        try:
            likes = post['likes']['count'] #собираем лайки
        except:
            likes = 0
        
        try:
            reposts = post['reposts']['count'] #собираем репосты
        except:
            reposts = 0
        
        try:
            comments = post['comments']['count'] #собираем репосты
        except:
            comments = 0
        
        try:
            views = post['views']['count'] #собираем кол-во просмотров
        except:
            views = 0
        
        try:
            text = post['text'] #собираем текст
        except:
            text = ''
            
        filtered_post = {
            'date': date,
            'timestamp': timestamp,
            'likes': likes,
            'reposts': reposts,
            'comments': comments,
            'views': views,
            'text': text
            }
        
        cleaned_data.append(filtered_post)
    
    return cleaned_data

In [10]:
#применяем функцию
all_posts_clean = clean_posts(all_posts)

In [11]:
all_posts_clean[:10]

[{'date': '18-12-2020 16:30:01',
  'timestamp': 1608291001,
  'likes': 72,
  'reposts': 4,
  'comments': 0,
  'views': 585666,
  'text': 'В «Подслушано» открыт эксклюзивный закрытый клуб 🤪 \n \nМы ищем Донов. Стань Доном и войди в ряды людей с супер способностями. К примеру, Доны могут комментировать секреты и читать закрытые от посторонних глаз записи в «Подслушано». \n \nДобро пожаловать! \n \nПодробнее в блоке «Поддержать автора» внутри [club34215577|нашего сообщества].'},
 {'date': '27-12-2020 16:30:02',
  'timestamp': 1609068602,
  'likes': 31,
  'reposts': 0,
  'comments': 0,
  'views': 1479,
  'text': 'После смерти любимого человека у меня на руках — ехала крыша, была аппатия и депрессия. Психологи меня раздражали, читала сама соответствующую литературу, где было всё, что они мне пытались донести. И я нашла для себя лекарство — заниматься работой, чтобы лучше всех; отдых только активный, чтобы была постоянная движуха. И я живу для всех очень ярко, многие завидуют моей энергии, в

In [12]:
#сохраняем в json
#with open('podslushano_clean_posts.json', 'w') as f:
    #json.dump(all_posts_clean, f)

In [213]:
#создаем на всякий случай dataframe
#df = pd.read_json('podslushano_clean_posts.json')

In [214]:
df[:3]

,date,timestamp,likes,reposts,comments,views,text
0,2020-12-18 16:30:01,2020-12-18 11:30:01,72,4,0,585666,В «Подслушано» открыт эксклюзивный закрытый кл...
1,2020-12-27 16:30:02,2020-12-27 11:30:02,31,0,0,1479,После смерти любимого человека у меня на руках...
2,2020-12-27 16:05:00,2020-12-27 11:05:00,269,18,1,18469,"На улице встретила свою бывшую одноклассницу, ..."


In [215]:
df['year'] = pd.DatetimeIndex(df['date']).year

In [216]:
df.sample(3)

,date,timestamp,likes,reposts,comments,views,text,year
45111,2018-05-16 23:40:03,2018-05-16 18:40:03,6570,16,0,311934,После вторых родов набрала 26 лишних кг. 26! В...,2018
62743,2017-08-04 16:55:00,2017-04-08 11:55:00,14602,13,0,521651,Моя мама буквально зависима от сайтов объявлен...,2017
97241,2014-12-29 23:40:00,2014-12-29 18:40:00,12346,71,0,0,У моей бабушки фамилия Кит. Когда она еще рабо...,2014


In [217]:
df['year'].unique()

array([2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012])

In [218]:
#выбираем только посты 2020 года
df_2020 = df[(df.year == 2020)].reset_index(drop=True) 

In [219]:
df_2020.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16930 entries, 0 to 16929
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   date       16930 non-null  datetime64[ns]
 1   timestamp  16930 non-null  datetime64[ns]
 2   likes      16930 non-null  int64         
 3   reposts    16930 non-null  int64         
 4   comments   16930 non-null  int64         
 5   views      16930 non-null  int64         
 6   text       16930 non-null  object        
 7   year       16930 non-null  int64         
dtypes: datetime64[ns](2), int64(5), object(1)
memory usage: 1.0+ MB


In [220]:
df_2020.head(3)

,date,timestamp,likes,reposts,comments,views,text,year
0,2020-12-18 16:30:01,2020-12-18 11:30:01,72,4,0,585666,В «Подслушано» открыт эксклюзивный закрытый кл...,2020
1,2020-12-27 16:30:02,2020-12-27 11:30:02,31,0,0,1479,После смерти любимого человека у меня на руках...,2020
2,2020-12-27 16:05:00,2020-12-27 11:05:00,269,18,1,18469,"На улице встретила свою бывшую одноклассницу, ...",2020


In [222]:
#safe to csv
#df_2020.to_csv('df_2020_filtered.csv', index=False)